Импорт библиотек и модулей (запускаем всегда)

In [1]:
from datetime import datetime, timedelta
from dateutil import parser

from settings.config import *
from utils.common import *

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [18]:
months = {
    'янв': 'Jan', 'фев': 'Feb', 'мар': 'Mar', 'апр': 'Apr',
    'мая': 'May', 'июн': 'Jun', 'июл': 'Jul', 'авг': 'Aug',
    'сен': 'Sep', 'окт': 'Oct', 'ноя': 'Nov', 'дек': 'Dec'
}

def parse_date(date_str):
    if 'сегодня' in date_str:
        parsed_date = datetime.now().date()

    elif 'вчера' in date_str:
        parsed_date = datetime.now().date() - timedelta(days=1)

    else:
        if ' в ' in date_str:  # Формат '15 мая в 19:17'
            current_year = datetime.now().year
            date_str = f"{date_str.split(' в ')[0]} {current_year}"  # Добавляем текущий год

        for ru_month, en_month in months.items():
            if ru_month in date_str:
                date_str = date_str.replace(ru_month, en_month)
                break
            
        parsed_date = parser.parse(date_str, dayfirst=True).date()

    return parsed_date

In [2]:
def find_error(driver):
    try:
        error_window = WebDriverWait(driver, WAIT_TIME * 0.2).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='box_layout']"))
        )
        if 'Ошибка' not in error_window.text:
            return False
        close_button = WebDriverWait(error_window, WAIT_TIME).until(
            EC.element_to_be_clickable((By.XPATH, ".//span[text()='Закрыть']"))
        )
        click_safely(close_button)
        return True
    
    except (TimeoutException, Exception):
        return False

Загрузка файла (запускаем всегда)

In [29]:
excel_file = 'to_delete.xlsx'

names_to_delete = pd.read_excel(excel_file, header=None)
names_to_delete

,0
0,Венера
1,Виола
2,Виолетта
3,Гузяль
4,Люба
...,...
214,Ники
215,Никос
216,Сергиус
217,Асхад


СТАВИМ НЕОБХОДИМУЮ ДАТУ (год, месяц, день)

In [4]:
current_date_f = datetime(2025, 4, 30).date()

Загрузка браузера (запускаем всегда)

ЗАХОДИМ В VK ЛЮБЫМ УДОБНЫМ СПОСОБОМ!

In [34]:
driver = load_webdriver()
load_website(driver, 'https://vk.com')

Браузер запущен
Страница 'https://vk.com' загружена


True

НАХОДИМ НУЖНЫЙ КАБИНЕТ И ПЕРЕХОДИМ ВО ВКЛАДКУ "РЕТАРГЕТИНГ"

In [ ]:
# url = 'https://vk.com/ads?act=retargeting&union_id=1608413111'  # 1 Полина
# url = 'https://vk.com/ads?act=retargeting&union_id=1608426792'  # 5 Полина2
# url = 'https://vk.com/ads?act=retargeting&union_id=1608429820'  # 4 Полина3
# url = 'https://vk.com/ads?act=retargeting&union_id=1608443472'  # 1 Полина4
url = 'https://vk.com/ads?act=retargeting&union_id=1608454152'  # 3 Полина 5

# url = 'https://vk.com/ads?act=retargeting&union_id=1608460747'  # 2 Полина 7
# url = 'https://vk.com/ads?act=retargeting&union_id=1608399616'  # 1 Полина 8
# url = 'https://vk.com/ads?act=retargeting&union_id=1608467167'  # 5 Полина 9
# url = 'https://vk.com/ads?act=retargeting&union_id=1608471786'  # 1 Полина 10

Основной цикл (запускаем первый раз и больше не запускаем)

In [31]:
load_website(driver, url)

deleted, not_found, with_error = [], [], []

for i, row in names_to_delete.iterrows():
    # По умолчанию начинаем с 0. Если нужно начать не с 0, то указываем стартовый индекс
    # Тогда строки до этого индекса (не включительно) будут пропущены
    if i < 0:
        continue

    name = row[0].split(' ')[0]

    while True:
        try:
            search_input = WebDriverWait(driver, WAIT_TIME).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@class='ui_search_field _field']"))
            )
            insert_value_safely(driver, search_input, name)
            time.sleep(SLEEP_TIME * 0.5)

            auditory_poles = WebDriverWait(driver, WAIT_TIME * 0.5).until(
                EC.presence_of_all_elements_located((By.XPATH, f".//tr[contains(@class, 'ui_table_row')]"))
            )

            is_found, is_deleted = False, False
            for auditory_pole in auditory_poles[1:21]:
                if len(auditory_pole.text) == 0:
                    continue

                auditory_element = auditory_pole.find_element(By.XPATH, ".//a[@class='ads_retargeting_group_actions_link']")
                if auditory_element.text != name:
                    continue

                auditory_date = auditory_pole.find_element(By.XPATH, ".//span[@class='ads_retargeting_group_wip']")
                date_f = parse_date(auditory_date.text)
                delta = (current_date_f - date_f).days
                if delta < 0:
                    continue

                is_found = True
                click_safely(auditory_element)

                auditory_functions_element = WebDriverWait(driver, WAIT_TIME).until(
                    EC.element_to_be_clickable((By.XPATH, f".//div[@class='dd_menu_rows2']"))
                )
                auditory_delete_button = WebDriverWait(auditory_functions_element, WAIT_TIME).until(
                    EC.element_to_be_clickable((By.XPATH, ".//a[text()='Удалить']"))
                )
                auditory_delete_button.click()
                time.sleep(SLEEP_TIME * 0.5)

                confirm_window = WebDriverWait(driver, WAIT_TIME).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='box_layout']"))
                )
                yes_button = WebDriverWait(confirm_window, WAIT_TIME).until(
                    EC.element_to_be_clickable((By.XPATH, ".//span[text()='Да']"))
                )
                click_safely(yes_button)

                if find_error(driver):
                    with_error.append(name)
                    print(i, name, date_f, 'Ошибка')
                    continue

                deleted.append(name)
                is_deleted = True
                print(i, name, date_f, 'Удалено')
                break
            
            if not is_found:
                if name not in deleted:
                    not_found.append(name)
                    print(i, name, 'Не найдено')

            if not is_deleted:
                break
        
        except (TimeoutException, Exception):
            not_found.append(name)
            print(i, name, 'Не найдено')
            continue

Страница 'https://vk.com/ads?act=retargeting&union_id=1608454152' загружена
0 Венера Не найдено
1 Виола 2025-02-21 Удалено
2 Виолетта 2025-02-22 Удалено
3 Гузяль 2025-02-20 Удалено
4 Люба 2025-02-22 Удалено
5 Анар Не найдено
5 Анар 2025-02-23 Удалено
6 Амнат 2025-02-23 Удалено
7 Гульжан 2025-02-23 Удалено
8 Гульзада 2025-02-22 Удалено
9 Гульзира 2025-02-23 Удалено
10 Гульсара 2025-02-23 Удалено
11 Дианна 2025-02-21 Удалено
12 Аленочка 2025-02-20 Удалено
13 Алвард 2025-02-23 Удалено
14 Айя 2025-02-23 Удалено
15 Джемма 2025-02-19 Удалено
16 Закия 2025-02-19 Удалено
17 Милана 2025-02-18 Удалено
18 Бозорова 2025-03-09 Удалено
19 Боженова 2025-03-09 Удалено
20 Богатырь 2025-03-09 Удалено
21 Бобылева 2025-03-09 Удалено
22 Бобрикова 2025-03-09 Удалено
23 Блюм 2025-03-09 Удалено
24 Беспалова 2025-03-09 Удалено
25 Юнг 2025-03-08 Удалено
26 Юровских 2025-03-08 Удалено
27 Юрчук 2025-03-08 Удалено
28 Юшина 2025-03-08 Удалено
29 Юшкова 2025-03-08 Удалено
30 Ягафарова 2025-03-08 Удалено
31 Ягода 202

KeyboardInterrupt: 

In [ ]:
for n in not_found:
    print(n)

In [ ]:
for n in with_error:
    print(n)

In [32]:
excel_file = 'to_delete.xlsx'

names_to_delete = pd.read_excel(excel_file, header=None)
names_to_delete

,0
0,Кеша
1,Лео
2,Лин
3,Лутфулло
4,Маис
...,...
214,Ники
215,Никос
216,Сергиус
217,Асхад


In [33]:
url = 'https://vk.com/ads?act=retargeting&union_id=1608399616'  # 1 Полина 8

In [35]:
load_website(driver, url)

deleted, not_found, with_error = [], [], []

for i, row in names_to_delete.iterrows():
    # По умолчанию начинаем с 0. Если нужно начать не с 0, то указываем стартовый индекс
    # Тогда строки до этого индекса (не включительно) будут пропущены
    if i < 0:
        continue

    name = row[0].split(' ')[0]

    while True:
        try:
            search_input = WebDriverWait(driver, WAIT_TIME).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@class='ui_search_field _field']"))
            )
            insert_value_safely(driver, search_input, name)
            time.sleep(SLEEP_TIME * 0.5)

            auditory_poles = WebDriverWait(driver, WAIT_TIME * 0.5).until(
                EC.presence_of_all_elements_located((By.XPATH, f".//tr[contains(@class, 'ui_table_row')]"))
            )

            is_found, is_deleted = False, False
            for auditory_pole in auditory_poles[1:21]:
                if len(auditory_pole.text) == 0:
                    continue

                auditory_element = auditory_pole.find_element(By.XPATH, ".//a[@class='ads_retargeting_group_actions_link']")
                if auditory_element.text != name:
                    continue

                auditory_date = auditory_pole.find_element(By.XPATH, ".//span[@class='ads_retargeting_group_wip']")
                date_f = parse_date(auditory_date.text)
                delta = (current_date_f - date_f).days
                if delta < 0:
                    continue

                is_found = True
                click_safely(auditory_element)

                auditory_functions_element = WebDriverWait(driver, WAIT_TIME).until(
                    EC.element_to_be_clickable((By.XPATH, f".//div[@class='dd_menu_rows2']"))
                )
                auditory_delete_button = WebDriverWait(auditory_functions_element, WAIT_TIME).until(
                    EC.element_to_be_clickable((By.XPATH, ".//a[text()='Удалить']"))
                )
                auditory_delete_button.click()
                time.sleep(SLEEP_TIME * 0.5)

                confirm_window = WebDriverWait(driver, WAIT_TIME).until(
                    EC.presence_of_element_located((By.XPATH, "//div[@class='box_layout']"))
                )
                yes_button = WebDriverWait(confirm_window, WAIT_TIME).until(
                    EC.element_to_be_clickable((By.XPATH, ".//span[text()='Да']"))
                )
                click_safely(yes_button)

                if find_error(driver):
                    with_error.append(name)
                    print(i, name, date_f, 'Ошибка')
                    continue

                deleted.append(name)
                is_deleted = True
                print(i, name, date_f, 'Удалено')
                break
            
            if not is_found:
                if name not in deleted:
                    not_found.append(name)
                    print(i, name, 'Не найдено')

            if not is_deleted:
                break
        
        except (TimeoutException, Exception):
            not_found.append(name)
            print(i, name, 'Не найдено')
            continue

Страница 'https://vk.com/ads?act=retargeting&union_id=1608399616' загружена
0 Кеша 2025-04-21 Удалено
1 Лео 2025-04-21 Удалено
2 Лин Не найдено
3 Лутфулло 2025-04-21 Удалено
4 Маис 2025-04-21 Удалено
5 Маркус Не найдено
6 Валиджон 2025-04-18 Удалено
7 Мики 2025-04-17 Удалено
8 Мирас 2025-04-17 Удалено
9 Мират 2025-04-17 Удалено
10 Мирбек 2025-04-17 Удалено
11 Миро 2025-04-17 Удалено
12 Михайл 2025-04-17 Удалено
13 Михан 2025-04-17 Удалено
14 Мухсин 2025-04-17 Удалено
15 Кемран 2025-04-22 Удалено
16 Зигмунд 2025-04-23 Удалено
17 Игнат 2025-04-23 Удалено
18 Идель 2025-04-23 Удалено
19 Икрам 2025-04-23 Удалено
20 Илес 2025-04-22 Удалено
21 Илхомжон 2025-04-22 Удалено
22 Ильдус 2025-04-22 Удалено
23 Ильюха 2025-04-22 Удалено
24 Иляс 2025-04-22 Удалено
25 Иннокентий 2025-04-22 Удалено
26 Ион 2025-04-22 Удалено
27 Ираклий 2025-04-22 Удалено
28 Ирик 2025-04-22 Удалено
29 Исак 2025-04-22 Удалено
30 Исо 2025-04-22 Удалено
31 Кади 2025-04-22 Удалено
32 Кадир 2025-04-22 Удалено
33 Кан 2025-04-22 

Дополнительный цикл (запускаем сколько угодно раз, если есть неуспешные записи)

In [ ]:
driver.refresh()

deleted_2, not_found_2, with_error_2 = [], [], []

for i, name in enumerate(not_found):

    try:
        search_input = WebDriverWait(driver, WAIT_TIME).until(
            EC.element_to_be_clickable((By.XPATH, "//input[@class='ui_search_field _field']"))
        )
        insert_value_safely(driver, search_input, name)
        time.sleep(SLEEP_TIME * 0.5)

        auditory_element = WebDriverWait(driver, WAIT_TIME).until(
            EC.element_to_be_clickable((By.XPATH, f".//a[@class='ads_retargeting_group_actions_link'][text()='{name}']"))
        )
        click_safely(auditory_element)

        auditory_functions_element = WebDriverWait(driver, WAIT_TIME).until(
            EC.element_to_be_clickable((By.XPATH, f".//div[@class='dd_menu_rows2']"))
        )
        auditory_delete_button = WebDriverWait(auditory_functions_element, WAIT_TIME).until(
            EC.element_to_be_clickable((By.XPATH, ".//a[text()='Удалить']"))
        )
        auditory_delete_button.click()
        time.sleep(SLEEP_TIME * 0.5)

        confirm_window = WebDriverWait(driver, WAIT_TIME).until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='box_layout']"))
        )
        yes_button = WebDriverWait(confirm_window, WAIT_TIME).until(
            EC.element_to_be_clickable((By.XPATH, ".//span[text()='Да']"))
        )
        click_safely(yes_button)
        
        if find_error(driver):
            with_error_2.append(name)
            print(i, name, 'Ошибка')

        deleted_2.append(name)
        print(i, name, 'Удалено')
    
    except (TimeoutException, Exception):
        not_found_2.append(name)
        print(i, name, 'Не найдено')
        continue

In [ ]:
for n in not_found_2:
    print(n)